
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# <center>Tracing with LiteLLM</center>    

This guide demonstrates how to use Arize with LiteLLM for tracing. 

You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to trace your LLM calls with LiteLLM.

ℹ️ This notebook requires:
- A OpenAI API key
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [2]:
# Dependencies needed to instrument your openai application using opentelemetry and openinference
!pip3 install arize-otel openinference-instrumentation-litellm opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-httpx litellm 'httpx<0.28'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [3]:
import os
from getpass import getpass

if not (SPACE_ID := os.getenv("SPACE_ID")):
    SPACE_ID = getpass("🔑 Enter your Space ID: ")

if not (API_KEY := os.getenv("API_KEY")):
    API_KEY = getpass("🔑 Enter your API Key: ")

if not (OPENAI_API_KEY := os.getenv("OPENAI_API_KEY")):
    OPENAI_API_KEY = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Step 3. Add our tracing code
We will be using the arize_otel package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [4]:
# Import open-telemetry dependencies
from arize_otel import register_otel, Endpoints
from openinference.instrumentation.litellm import LiteLLMInstrumentor

# Setup OTEL via our convenience function
register_otel(
    endpoints=Endpoints.ARIZE,
    space_id=SPACE_ID,  # in app space settings page
    api_key=API_KEY,  # in app space settings page
    project_name="litellm-tracing-tutorial",  # name this to whatever you would like
)

LiteLLMInstrumentor().instrument()

## Step 4: Run your LLM application
Let's test our app by asking to write a haiku.

In [6]:
import litellm

litellm.completion(
    model="gpt-3.5-turbo",
    messages=[{"content": "Write a haiku.", "role": "user"}],
)

ModelResponse(id='chatcmpl-AbDKfttNH7Q6kfQ4wIcecr4qKNj8Y', created=1733432749, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Gentle breeze whispers\nThrough the leaves of the cherry tree\nNature's sweet lullaby", role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=20, prompt_tokens=12, total_tokens=32, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)


Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`.

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/arize-llm-tracing.png" width="1500" />